# Plot paired and unpaired synapses

In [2]:
# Load up all of the libraries we need

from deriva.core import HatracStore, ErmrestCatalog, get_credential
from synspy.analyze.pair import SynapticPairStudy

import synapse_utils

import pandas as pd
import numpy as np

import plotly as py
import plotly.graph_objs as go

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [3]:
credential = get_credential("synapse.isrd.isi.edu")
ermrest_catalog = ErmrestCatalog('https', 'synapse.isrd.isi.edu', 1, credential)
hatrac_store = HatracStore('https', 'synapse.isrd.isi.edu', credentials=credential)

Lets start by getting a list of all of the studies currently in the server. We will end up with a list of studies, where each study has been classified by the study type based on the protocol that was followed. 

In [8]:
# Get the current list of studies from the server.
study_entities = synapse_utils.get_synapse_studies()

print('Identified %d studies' % len(study_entities))

protocol_types = {
    'PrcDsy20160101A': 'aversion',
    'PrcDsy20170613A': 'conditioned',
    'PrcDsy20170615A': 'unconditioned',
    'PrcDsy20170613B': 'fullcycle-control',
    'PrcDsy20171030A': 'groundtruth-control',
    'PrcDsy20171030B': 'interval-groundtruth-control'
}

studies={}
for i in study_entities:
    studies[i['Study']] = {}
    if protocol_types[i['Protocol']] == 'aversion':
        studies[i['study']]['type'] = 'learner' if i['Learner'] == True else 'nonlearner'
    else:
        studies[i['study']]['type'] = protocol_types[i['Protocol']]

Identified 65 studies


Lets make some convienent subsets of studies

In [33]:
learners=[ x for x,y in studies.items() if y['Type'] == 'learner' ]
nonlearners = [ x for x,y in studies.items() if y['Type'] == 'nonlearner' ]
all = [ x for x in studies]

print('{0} learners'.format(len(learners)))
print('{0} non-learners'.format(len(nonlearners)))

18 learners
13 non-learners


Pick a study that we want to look at and set the parameters for the sweep.

In [20]:
syn_pair_radii = (2.0, 3.0, 4.0) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

Now calculate the distances and save the results into dataframes....

In [21]:
for syn_study_id in studies:
    print('Processing study {0}'.format(syn_study_id))
    study = SynapticPairStudy.from_study_id(ermrest_catalog, syn_study_id)
    study.retrieve_data(hatrac_store)
    
    studies[syn_study_id]['count'] = len(study.s1)
    studies[syn_study_id]['unpaired_before'] = {}
    studies[syn_study_id]['unpaired_after'] = {}
    studies[syn_study_id]['paired_before'] = {}
    studies[syn_study_id]['paired_after'] = {}
    
    s1_to_s2, s2_to_s1 = study.syn_pairing_maps(syn_pair_radii, syn_dx_core_ratio, syn_core_max_ratio)

    # get results for different radii, store them in a dictonary of pandas
    for i,r in enumerate(syn_pair_radii):
        unpaired1 = study.get_unpaired(s1_to_s2[i,:], study.s1)
        unpaired2 = study.get_unpaired(s2_to_s1[i,:], study.s2)
        paired1, paired2 = study.get_pairs(s1_to_s2[i,:], study.s1, study.s2)
    
 
        studies[syn_study_id]['unpaired_before'][r] =  \
            pd.DataFrame(unpaired1[:,0:5], columns=['x','y','z','core','hollow'])
        studies[syn_study_id]['unpaired_after'][r] = \
            pd.DataFrame(unpaired2[:,0:5], columns=['x','y','z','core','hollow'])
        studies[syn_study_id]['paired_before'][r] = \
            pd.DataFrame(paired1[:,0:5], columns=['x','y','z','core','hollow'])
        studies[syn_study_id]['paired_after'][r] = \
            pd.DataFrame(paired2[:,0:5], columns=['x','y','z','core','hollow'])

Processing study SynStd6025
Processing study SynStd6061
Processing study SynStd6471
Processing study SynStd6473
Processing study SynStd6475
Processing study SynStd6477
Processing study SynStd6479
Processing study SynStd6481
Processing study SynStd6483
Processing study SynStd6485
Processing study SynStd6487
Processing study SynStd6489
Processing study SynStd6491
Processing study SynStd6493
Processing study SynStd6495
Processing study SynStd6687
Processing study SynStd6689
Processing study SynStd6697
Processing study SynStd6701
Processing study SynStd6763
Processing study SynStd6791
Processing study SynStd6823
Processing study SynStd6957
Processing study SynStd7633
Processing study SynStd7635
Processing study SynStd7637
Processing study SynStd7941
Processing study SynStd7943
Processing study SynStd7945
Processing study SynStd7947
Processing study SynStd7949
Processing study SynStd7951
Processing study SynStd7955
Processing study SynStd7957
Processing study SynStd7963
Processing study Syn

Compute some basic Statistics

ValueError: not enough values to unpack (expected 2, got 0)

In [56]:
cols = ['ID', 'Type', 'Total', 'Radius', 'Unpaired Before', 'Unpaired After','Paired Before','Paired After']
c1, c2, c3, c4, c5, c6, c7, c8 = [],[],[],[],[],[],[],[]

for id,s in studies.items():
    for r in syn_pair_radii:
        c1.append(id)
        c2.append(s['Type'])
        c3.append(s['count'])
        c4.append(r)
        c5.append(len(s['unpaired_before'][r].index))
        c6.append(len(s['unpaired_after'][r].index))
        c7.append(len(s['paired_before'][r].index))
        c8.append(len(s['paired_before'][r].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6, c7, c8],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'pandas_table')

In [57]:
r = 4.0

#calculate centroids
learn_before_x = [ studies[s]['unpaired_before'][r]['x'].mean() for s in learners]
learn_before_y = [ studies[s]['unpaired_before'][r]['y'].mean() for s in learners]
learn_before_z = [ studies[s]['unpaired_before'][r]['z'].mean() for s in learners]

learn_after_x = [ studies[s]['unpaired_after'][r]['x'].mean() for s in learners]
learn_after_y = [ studies[s]['unpaired_after'][r]['y'].mean() for s in learners]
learn_after_z = [ studies[s]['unpaired_after'][r]['z'].mean() for s in learners]

non_before_x = [ studies[s]['unpaired_before'][r]['x'].mean() for s in nonlearners]
non_before_y = [ studies[s]['unpaired_before'][r]['y'].mean() for s in nonlearners]
non_before_z = [ studies[s]['unpaired_before'][r]['z'].mean() for s in nonlearners]

non_after_x = [ studies[s]['unpaired_after'][r]['x'].mean() for s in nonlearners]
non_after_y = [ studies[s]['unpaired_after'][r]['y'].mean() for s in nonlearners]
non_after_z = [ studies[s]['unpaired_after'][r]['z'].mean() for s in nonlearners]

learner_bcentroids = go.Scatter3d(
        x=learn_before_x, y=learn_before_y, z=learn_before_z,
        mode='markers',
        name='Learner Before centroids',
        text=learners,
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

learner_acentroids = go.Scatter3d(
        x=learn_after_x, y=learn_after_y, z=learn_after_z,
        mode='markers',
        name='Learner After centroids',
        text=learners,
        marker=dict(
            size=2,
            line=dict(
            color='rgb(255,0,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

nonlearner_bcentroids = go.Scatter3d(
        x=non_before_x, y=non_before_y, z=non_before_z,
        mode='markers',
        name='Nonlearner Before centroids',
        text=nonlearners,
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )

nonlearner_acentroids = go.Scatter3d(
        x=non_after_x, y=non_after_y, z=non_after_z,
        mode='markers',
        name='Nonlearner After centroids',
        text=nonlearners,
        marker=dict(
            size=2,
            line=dict(
            color='rgb(128,128,128            )',
            width=0.5
            ),
        opacity=0.5
        )
    )

data = [learner_bcentroids, learner_acentroids, nonlearner_acentroids, nonlearner_bcentroids]
py.offline.iplot(data, filename='pair-distance')


Plot histograms of different radii

In [ ]:
data = []

r = max(syn_pair_radii)
before = paired_before[r]
after = paired_after[r]
pair_distance = np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)
                
hist_data = go.Histogram(x=pair_distance, name='Radias {0}'.format(r))

hist_layout = go.Layout(
    title='Pair Histogram Radius {0}: {1} ({2})'.format(r, study.id, study_types[study.id]),
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data= [hist_data], layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

In [ ]:
data = []

# A helper function to set visiblity     

for r in  syn_pair_radii:
    
    before = paired_before[r]
    after = paired_after[r]
    up_before = unpaired_before[r]
    up_after = unpaired_after[r]
  
    before_synapses = go.Scatter3d(
        visible = False,
        x=before['x'], y=before['y'], z=before['z'],
        legendgroup='paired',
        name='Before Paired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    after_synapses = go.Scatter3d(
        visible = False,
        x=after['x'], y=after['y'], z=after['z'],
        legendgroup='paired',
        name='After Paired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.5
        )
    )
    
    up_before_synapses = go.Scatter3d(
        visible = False,      
        x=up_before['x'], y=up_before['y'], z=up_before['z'],
        legendgroup='unpaired',
        name='Before Unpaired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    up_after_synapses = go.Scatter3d(
        visible = False,
        x=up_after['x'], y=up_after['y'], z=up_after['z'],
        legendgroup='unpaired',
        name='After Unpaired Radius: {0}'.format(r),
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

    data.extend([ before_synapses, after_synapses, up_before_synapses, up_after_synapses])

for i in range(4):
    data[i]['visible'] = True

steps = []
for i in range(len(data)//4):
    step = dict(
        method = 'restyle',
        args = ['visible', [False] * len(data)],
        label='Radius {0}'.format(syn_pair_radii[i])
    )
    step['args'][1][i*4:i+4] = [True] * 4 # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = 0,
    currentvalue = {"visible": False},
    pad = {"t": 50},
    steps = steps
)]

layout = go.Layout(
        height=1000, width=1000,
        title='Paired and Unpaired Synapapses {0} ({1})'.format(study.id, study_types[study.id]),
        sliders=sliders
    )

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='Synapse Pair Distance')

In [ ]:
for r in  syn_pair_radii:
    
    before = paired_before[r]
    after = paired_after[r]
    up_before = unpaired_before[r]
    up_after = unpaired_after[r]
  
    before_synapses = go.Scatter3d(
        x=before['x'], y=before['y'], z=before['z'],
        legendgroup='paired',
        name='Before Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    after_synapses = go.Scatter3d(
        x=after['x'], y=after['y'], z=after['z'],
        legendgroup='paired',
        name='After Paired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.5
        )
    )
    
    up_before_synapses = go.Scatter3d(
        x=up_before['x'], y=up_before['y'], z=up_before['z'],
        legendgroup='unpaired',
        name='Before Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,0,255)',
            width=0.5
            ),
        opacity=0.5
        )
    )

    up_after_synapses = go.Scatter3d(
        x=up_after['x'], y=up_after['y'], z=up_after['z'],
        legendgroup='unpaired',
        name='After Unpaired',
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(128,128,128)',
                width=.5
            ),
        opacity=0.5
        )
    )

    data = [ before_synapses, after_synapses, up_before_synapses, up_after_synapses]

    layout = go.Layout(
        height=800,
        title='Paired Synapapses at Radius {0}'.format(r),
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)